# TP4
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Exercício 4.1

### Descrição do Problema

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    
1. A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. No modo `Free`  não existe qualquer força de travagem; no modo `Stopping` aplica-se a força de travagem alta; no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  desloca-se; no modo `Stopped` o veículo está imobilizado.
2. A componente contínua  do autómato usa variáveis contínuas $\,V,v\,$ para descrever a  `velocidade do corpo`  do veículo em relação ao solo e a `velocidade linear das rodas` também em relação ao solo. 
        Assume-se que o sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades.  A dinâmica contínua está descrita  abaixo no bloco Equaçoes de Fluxo.
3. Os “switchs” (“jumps”) são a componente de projeto deste trabalho; cabe ao aluno definir quais devem ser estas  condições de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
4. É imprescindível evitar que o sistema tenha “trajetórias de Zenão”. Isto é, sequências  infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.

### Abordagem do Problema

1. Componentes Contínuas
velocidade linear das rodas
velocidade do corpo (veículo)
tempo?
Força de travagem

2. Componentes Discretas
força de atrito do solo




Nodos: Start, Free, Stopping, Blocked e Stopped
Start -> Inicio
Free -> A força de travagem não é exercida sobre o veículo
Stopping -> É aplicada uma força de travagem alta sobre o veículo
Blocked -> As rodas estão travadas, porém o veículo continua a mover-se
Stopped -> O véiculo para

Jumps (ter em atenção o comportamento desejável: imobilize-se depressa e não “derrape” muito):
Start -> Free (Inicia para um movimento livre)
Start -> Stopping (Logo no ínicio, veículo começa a ação de paragem com a ajuda dos travões)
Start -> Stopped (O veículo consegue parar sem o auxilio dos travões)
Stopping -> Blocked (O véiculo começa a derrapar)
Blocked -> Stopped (Após derrapar, o veículo finalmente para)
Free -> Stopping (Começa a ação de travagem)
Stopping -> Free (?)
(Além de mais possíveis jumps, falta declarar as condições de cada um)



## Código Python